In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = 15,7
import seaborn as sns

train = pd.read_csv('./train.csv')
train['Tag'] = train.Tag.astype('category')

In [2]:
#DA
print(len(train.ID))
print(len(train.ID.unique()))
train.isna().sum()

330045
330045


ID            0
Tag           0
Reputation    0
Answers       0
Username      0
Views         0
Upvotes       0
dtype: int64

In [3]:
#Preprocessing data
#Converting categorical variables
df = pd.get_dummies(train)
df.drop('Tag_a', axis = 1, inplace = True)
df.head()

,ID,Reputation,Answers,Username,Views,Upvotes,Tag_c,Tag_h,Tag_i,Tag_j,Tag_o,Tag_p,Tag_r,Tag_s,Tag_x
0,52664,3942.0,2.0,155623,7855.0,42.0,0,0,0,0,0,0,0,0,0
1,327662,26046.0,12.0,21781,55801.0,1175.0,0,0,0,0,0,0,0,0,0
2,468453,1358.0,4.0,56177,8067.0,60.0,1,0,0,0,0,0,0,0,0
3,96996,264.0,3.0,168793,27064.0,9.0,0,0,0,0,0,0,0,0,0
4,131465,4271.0,4.0,112223,13986.0,83.0,1,0,0,0,0,0,0,0,0


In [4]:
df.columns

Index(['ID', 'Reputation', 'Answers', 'Username', 'Views', 'Upvotes', 'Tag_c',
       'Tag_h', 'Tag_i', 'Tag_j', 'Tag_o', 'Tag_p', 'Tag_r', 'Tag_s', 'Tag_x'],
      dtype='object')

In [5]:
#Scaling of features
features = ['Reputation', 'Answers', 'Username', 'Views', 'Tag_c', 'Tag_h', 'Tag_i',\
            'Tag_j', 'Tag_o', 'Tag_p', 'Tag_r', 'Tag_s', 'Tag_x']
numeric_features = ['Reputation', 'Answers', 'Username', 'Views']
X = df[features]
y = df['Upvotes']

In [6]:
df[numeric_features].describe()

,Reputation,Answers,Username,Views
count,3.300450e+05,330045.000000,330045.000000,3.300450e+05
mean,7.773147e+03,3.917672,81442.888803,2.964507e+04
std,2.706141e+04,3.579515,49215.100730,8.095646e+04
min,0.000000e+00,0.000000,0.000000,9.000000e+00
25%,2.820000e+02,2.000000,39808.000000,2.594000e+03
50%,1.236000e+03,3.000000,79010.000000,8.954000e+03
75%,5.118000e+03,5.000000,122559.000000,2.687000e+04
max,1.042428e+06,76.000000,175738.000000,5.231058e+06


In [7]:
from sklearn.preprocessing import MaxAbsScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE
from sklearn.model_selection import cross_val_score

import numpy as np

pipeline = Pipeline([('scaler', MaxAbsScaler() ), \
                     ('rf', RandomForestRegressor(n_estimators = 400, min_samples_leaf=0.12, random_state = 23) )])

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 23)

rf_scaled = pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
rmse = MSE(y_test, y_pred) ** 0.5
print(rmse)

test = pd.read_csv('./test.csv')
test = test.sort_values(by = 'ID')
test['Tag'] = test['Tag'].astype('category')
test = pd.get_dummies(test)
test.drop('Tag_a', axis = 1, inplace = True)
test_X = test[features]
y_pred_test = pipeline.predict(test_X)
predictions_df = pd.DataFrame({'ID': np.array(test['ID']), 'Upvotes':np.array(y_pred_test)})
predictions_df.to_csv('./submission.csv', index=False)

3509.4350085101282
   ID      Upvotes
0   7   240.563207
1   9    24.273888
2  12   240.563207
3  13  1437.739472
4  16    50.444416
